In [1]:
import cv2
import numpy as np
import warnings  # Ignore warnings for pyproj.

warnings.simplefilter("ignore", category=FutureWarning)

from pyproj import Proj, transform

In [2]:
def crsToSrs(min_lat, min_lon, max_lat, max_lon):
    """
    ## Converts coordinates from EPSG:4326 to EPSG25832.

    ## Parameters:
        - x1, y1, x2, y2 coordinates of EPSG:4326 WGS 84 system.

    ## Returns:
       - minimum x coordinate, minimum y coordinate, maximum x coordinate, maximum y coordinate

    """
    min_x, min_y = transform(Proj("EPSG:4326"), Proj("EPSG:25832"), min_lat, min_lon)
    max_x, max_y = transform(Proj("EPSG:4326"), Proj("EPSG:25832"), max_lat, max_lon)
    return min_x, min_y, max_x, max_y

def getScaleFactor(coordinates,imgWidth,robotWidth):
    min_lat, min_lon, max_lat, max_lon = coordinates
    min_x, min_y, max_x, max_y = crsToSrs(min_lat, min_lon, max_lat, max_lon)
    areaWidth = max_x-min_x
    newWidth = areaWidth/robotWidth
    print(imgWidth/newWidth)
    scaleFactor = int(imgWidth/newWidth)
    print(scaleFactor)
    return scaleFactor


def downscale(image,scaleFactor):
    #Converting to binary
    rasterMapGrey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, rasterMapBinary = cv2.threshold(rasterMapGrey, 1, 255, cv2.THRESH_BINARY)

    #DImensions of downscaled map
    downscaleHeight = int(rasterMapBinary.shape[0] // scaleFactor+ 1)
    downscaleWidth = int(rasterMapBinary.shape[1] // scaleFactor+ 1)

    #Make a white map
    downscaleMap = np.ones((downscaleHeight, downscaleWidth), dtype=np.uint8)*255

    #Iterate over all pixels in rasterMap
    for i in range(0, rasterMapBinary.shape[0], scaleFactor):
        for j in range(0, rasterMapBinary.shape[1], scaleFactor):

            #Get all pixels in a block to compute one pixel. 
            block = rasterMapBinary[i:i+scaleFactor, j:j+scaleFactor]

            if np.any(block == 0):  #If any pixel is black.
                downscaleMap[i//scaleFactor, j//scaleFactor] = 0  #set pixel to black
            else: #If it does not contain any black. 
                downscaleMap[i//scaleFactor, j//scaleFactor] = 255  #set pixel to white
    return downscaleMap

def convertToNumpy(map):

    startPixel = None
    height, width = map.shape

    #Setting the start pixel to first encountered sidewalk node. 
    for i in range(height):
        for j in range(width):
            if map[i, j] == 0:
                startPixel = (i, j)
                break
        if startPixel is not None:
            break

    #Making a zero np array for the size. 
    npArray = np.zeros((height, width), dtype=int)

    #Translating the binary to np array. 
    for i in range(height):
        for j in range(width):
            if (i, j) == startPixel:
                npArray[i, j] = 2
            elif map[i, j] == 0:
                npArray[i, j] = 0
            else:
                npArray[i, j] = 1
                
    return npArray

def displayMap(downScaledMap):
    height, width = downScaledMap.shape[:2]

    # Resizing
    newWidth = 1000
    scaleFactor = newWidth / width

    #Calculating new height
    newheight = int(height * scaleFactor)

    #Resize  image
    imshow = cv2.resize(downScaledMap, (newWidth, newheight), interpolation=cv2.INTER_NEAREST)

    #Display
    cv2.imshow("downscaleMap", imshow)
    cv2.waitKey(0)  
    cv2.destroyAllWindows()
    


#### Testing downsizing

In [ ]:
testMap = cv2.imread(f"../../ITUData/Pathplanning/test.jpg")
downSized = downscale(testMap,2)

original_height, original_width = downSized.shape[:2]

# Set the target width to 1000 pixels
target_width = 1000

# Calculate the scaling factor
scale_factor = target_width / original_width

# Calculate the new height based on the scaling factor
target_height = int(original_height * scale_factor)

# Resize the image using nearest-neighbor interpolation
resized_image = cv2.resize(downSized, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

# Display the resized image
cv2.imshow('Resized Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Converting raster maps to np array for path planning.

In [18]:
maps = ['aarhus1','aarhus10']
robotWidth = 0.5
coordinates = [[56.15460857439466,10.192037972762382,56.15581114830297, 10.199668652079422],[56.16994914071068,10.201318214312574,56.1725284201622,10.20817583862707]]

for i, map in enumerate(maps):

    #Reading map
    rasterMap = cv2.imread(f"../../ITUData/Pathplanning/{map}.jpg")

    #Getting image width.
    _,imgWidth = rasterMap.shape[:2]

    #Calculating scale factor for resizing. 
    scaleFactor = getScaleFactor(coordinates[i],imgWidth,robotWidth)

    #Downscaling
    downscaledMap = downscale(rasterMap,scaleFactor)
    displayMap(downscaledMap)
    
    #Converting to np array
    npMap = convertToNumpy(downscaledMap)

    #Saving np array.
    np.save(f"maps/{map}.npy", npMap)
    

4.929604917720456
4
5.525825516628273
5


### Applying Complete Coverage Path Planning Algorithm 

In [6]:
import subprocess
result = subprocess.run(['python', 'src/coverage_test.py'], capture_output=True, text=True)
#print("output:", result.stdout)
#print("error:", result.stderr)

stdout: Map tested: aarhus1
+------------+-------------+--------+-------+-------+
| Heuristic  | Orientation | Found? | Steps | Cost  |
+------------+-------------+--------+-------+-------+
| HORIZONTAL |      >      |  True  |  410  | 70.60 |
| HORIZONTAL |      ^      |  True  |  410  | 70.70 |
| HORIZONTAL |      v      |  True  |  410  | 70.70 |
| HORIZONTAL |      <      |  True  |  410  | 70.90 |
| CHEBYSHEV  |      v      |  True  |  429  | 77.60 |
| CHEBYSHEV  |      <      |  True  |  429  | 77.70 |
| MANHATTAN  |      >      |  True  |  431  | 72.40 |
| MANHATTAN  |      ^      |  True  |  431  | 72.50 |
| CHEBYSHEV  |      >      |  True  |  431  | 77.90 |
| CHEBYSHEV  |      ^      |  True  |  431  | 78.00 |
| MANHATTAN  |      v      |  True  |  434  | 72.60 |
| MANHATTAN  |      <      |  True  |  434  | 72.70 |
|  VERTICAL  |      v      |  True  |  455  | 80.50 |
|  VERTICAL  |      <      |  True  |  455  | 80.60 |
|  VERTICAL  |      >      |  True  |  455  | 80.60 |
